In [11]:
!pip install -q torch==0.4.1 torchvision==0.2.1

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
fastai 1.0.60 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import CenterCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import BASNet

import numpy as np
from matplotlib import pyplot as plt
import cv2

In [13]:
def normPRED(d):
	ma = torch.max(d)
	mi = torch.min(d)

	dn = (d-mi)/(ma-mi)

	return dn

def save_output(image_name,pred,d_dir):

	predict = pred
	predict = predict.squeeze()
	predict_np = predict.cpu().data.numpy()

	im = Image.fromarray(predict_np*255).convert('RGB')
	img_name = image_name.split("/")[-1]
	image = io.imread(image_name)
	imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

	pb_np = np.array(imo)

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	imo.save(d_dir+imidx+'.png')

In [14]:
image_dir = './test_data/custom_test_images/'
prediction_dir = './test_data/custom_test_results/'
model_dir = './saved_models/basnet_bsi/basnet.pth'

img_name_list = glob.glob(image_dir + '*.jpg')

In [15]:
test_salobj_dataset = SalObjDataset(img_name_list = img_name_list, lbl_name_list = [],transform=transforms.Compose([RescaleT(256),ToTensorLab(flag=0)]))
test_salobj_dataloader = DataLoader(test_salobj_dataset, batch_size=1,shuffle=False,num_workers=1)

In [16]:
net = BASNet(3,1)
net.load_state_dict(torch.load(model_dir))
if torch.cuda.is_available():
	net.cuda()
net.eval()

BASNet(
  (inconv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (inbn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (inrelu): ReLU(inplace=True)
  (encoder1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   

In [17]:
for i_test, data_test in enumerate(test_salobj_dataloader):

	print("inferencing:",img_name_list[i_test].split("/")[-1])

	inputs_test = data_test['image']
	inputs_test = inputs_test.type(torch.FloatTensor)

	if torch.cuda.is_available():
		inputs_test = Variable(inputs_test.cuda())
	else:
		inputs_test = Variable(inputs_test)

	d1,d2,d3,d4,d5,d6,d7,d8 = net(inputs_test)

	# normalization
	pred = d1[:,0,:,:]
	pred = normPRED(pred)

	# save results to test_results folder
	save_output(img_name_list[i_test],pred,prediction_dir)

	del d1,d2,d3,d4,d5,d6,d7,d8

inferencing: sample7.jpg


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  # loss


inferencing: sample10.jpg
inferencing: sample6.jpg
inferencing: sample11.jpg
inferencing: sample12.jpg
inferencing: sample8.jpg
inferencing: sample9.jpg
inferencing: sample1.jpg


In [39]:
img_name_list = sorted(img_name_list)
print(img_name_list)

['./test_data/custom_test_images/sample1.jpg', './test_data/custom_test_images/sample10.jpg', './test_data/custom_test_images/sample11.jpg', './test_data/custom_test_images/sample12.jpg', './test_data/custom_test_images/sample6.jpg', './test_data/custom_test_images/sample7.jpg', './test_data/custom_test_images/sample8.jpg', './test_data/custom_test_images/sample9.jpg']


In [33]:
result_img_list = glob.glob(prediction_dir + '*.png')
result_img_list = sorted(result_img_list)
print(result_img_list)

['./test_data/custom_test_results/sample1.png', './test_data/custom_test_results/sample10.png', './test_data/custom_test_results/sample11.png', './test_data/custom_test_results/sample12.png', './test_data/custom_test_results/sample6.png', './test_data/custom_test_results/sample7.png', './test_data/custom_test_results/sample8.png', './test_data/custom_test_results/sample9.png']


In [38]:
for img_num in range(len(img_name_list)):
    
    image = cv2.imread(img_name_list[img_num])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    result_image = cv2.imread(result_img_list[img_num])
    result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
    
    new_img = np.where(result_image<[250,250,250],[254,254,254],image)
    new_img = new_img.astype('uint8')
    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite('./test_data/background_removed_images/result_img' + str(img_num) + '.jpg',new_img)